In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import optuna
import joblib

In [ ]:
# --- اتصال به Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# --- خواندن داده
file_path = '/content/drive/My Drive/My_data/XAUUSD_M15_2020_2024.csv'
df = pd.read_csv(file_path)
df.tail(2)

In [ ]:
# --- پیش‌پردازش
label_mapping = {'b': 0, 's': 1, 'n': 2}
df['Lable'] = df['Lable'].map(label_mapping).fillna(-1).astype(int)

feature_columns = ['Open', 'High', 'Low', 'Close', 'tenkan', 'kijun', 'senkou_a', 'senkou_b', 'spanb_high_index',
                   'spanb_low_index', 'kijun_104', 'komo_sen', 'komo']

columns_to_normalize = ['Open', 'High', 'Low', 'Close', 'tenkan', 'kijun', 'senkou_a', 'senkou_b', 'kijun_104']
df_normalized = df.copy()
scaler = MinMaxScaler()
df_normalized[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [ ]:
# ذخیره Scaler برای استفاده بعدی
joblib.dump(scaler, '/content/drive/My Drive/My_data/minmax_scaler_1.pkl')

In [ ]:
# --- ساخت داده‌های ورودی و خروجی
timesteps = 52
X, y = [], []
for i in range(timesteps - 1, len(df_normalized)):
    label = df_normalized['Lable'].iloc[i]
    if pd.notna(label) and label in [0, 1, 2]:
        sample = df_normalized[feature_columns].iloc[i - timesteps + 1:i + 1].values
        X.append(sample)
        y.append(label)

X = np.array(X)
y = np.array(y)

In [ ]:
# --- تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
# --- One-hot Encoding
y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

In [ ]:
# --- وزن‌دهی به کلاس‌ها
class_weights = {0: 1.0 / 0.27, 1: 1.0 / 0.38, 2: 1.0 / 0.35}
total = sum(class_weights.values())
class_weights = {k: v / total for k, v in class_weights.items()}
sample_weights = np.array([class_weights[int(i)] for i in y_train])

In [ ]:
# --- تعریف مدل
def create_model(trial):
    model = Sequential()
    model.add(Input(shape=(timesteps, len(feature_columns))))

    num_layers = trial.suggest_int("num_layers", 1, 3)
    for i in range(num_layers):
        units = trial.suggest_categorical(f"units_{i}", [32, 64, 128, 256])
        return_seq = i < num_layers - 1
        model.add(LSTM(units, return_sequences=return_seq))
        dropout_rate = trial.suggest_float(f"dropout_{i}", 0.2, 0.5)
        model.add(Dropout(dropout_rate))

    model.add(Dense(3, activation='softmax'))

    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop", "sgd", "nadam"])
    optimizer = {
        'adam': Adam(learning_rate),
        'rmsprop': RMSprop(learning_rate),
        'sgd': SGD(learning_rate),
        'nadam': Nadam(learning_rate)
    }[optimizer_name]

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# --- تابع هدف Optuna
def objective(trial):
    model = create_model(trial)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(X_train, y_train_onehot,
                        epochs=42,
                        batch_size=batch_size,
                        validation_data=(X_test, y_test_onehot),
                        sample_weight=sample_weights,
                        callbacks=[early_stopping],
                        verbose=0)

    val_loss, val_accuracy = model.evaluate(X_test, y_test_onehot, verbose=0)
    return val_accuracy

In [ ]:
# --- اجرای بهینه‌سازی
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=75)

In [ ]:
# --- نمایش بهترین هایپرپارامترها
print("\n✅ بهترین هایپرپارامترها:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

In [ ]:
#  Best Parameters
# num_layers: 3
# units_0: 256
# dropout_0: 0.24078446184580357
# units_1: 128
# dropout_1: 0.3664629266022855
# units_2: 64
# dropout_2: 0.29437184419705087
# learning_rate: 0.0016267447887427097
# optimizer: adam
# batch_size: 16

In [ ]:
# --- آموزش نهایی با بهترین مدل
best_trial = study.best_trial
final_model = create_model(best_trial)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

final_model.fit(X_train, y_train_onehot,
                epochs=50,
                batch_size=best_trial.params['batch_size'],
                validation_data=(X_test, y_test_onehot),
                sample_weight=sample_weights,
                callbacks=[early_stopping],
                verbose=1)

In [ ]:
# --- پیش‌بینی نهایی
y_pred = final_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_onehot, axis=1)

In [ ]:
# --- گزارش نهایی
print("\n📊 گزارش نهایی:")
print(classification_report(y_true_classes, y_pred_classes, target_names=['Buy', 'Sell', 'No Action']))


In [ ]:
# --- Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Buy', 'Sell', 'No Action'], yticklabels=['Buy', 'Sell', 'No Action'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# --- ذخیره مدل نهایی
final_model.save('/content/drive/My Drive/My_data/best_lstm_evolution_model_1.h5')